In [0]:
!pip install flask-ngrok
!pip install gluonnlp
!pip install mxnet
!pip install leven
!pip install mxboard
!pip install nltk==3.2.5
!pip install sacremoses
import nltk
nltk.download('perluniprops')
nltk.download('nonbreaking_prefixes')
!pip install PyPDF2


     |████████████████████████████████| 245kB 3.5MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.8.3-cp36-none-any.whl size=293539 sha256=0d828efc0e4f81b40eb1ca6804a8a1245f7410bdd6ddfe3d52ab29bcfa713d27
  Stored in directory: /root/.cache/pip/wheels/50/6e/32/521aa84da7f9ee725d3c9be0b5e0d771df659bf25da5929f6c
Successfully built gluonnlp
     |████████████████████████████████| 25.4MB 1.8MB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 163kB 5.3MB/s 
  Created wheel for leven: filename=leven-1.0.4-cp36-cp36m-linux_x86_64.whl size=54679 sha256=40ba4add53655f73668cc04473b6b8b97b7b83deb39f5309be5b97138e409c24
  Stored in directory: /root/.cache/pip/wheels/54/64/a5/439db671d666a50f3b3cebd2dcab3fbbab02785adf58e47552
Successfully built leven
     |████████████████████████████████| 51kB 1.9MB/s 
     |████████████████████████████████| 870kB 3.5MB/s 
  Created wheel

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir('drive/My Drive/Colab Notebooks/models')

In [0]:
import difflib
import importlib
import math
import cv2 as cv2
import numpy as np
import mxnet as mx
import random
import matplotlib.pyplot as plt
import gluonnlp as nlp
import leven
import matplotlib.patches as patches
from skimage import transform as skimage_tf, exposure
from tqdm import tqdm
from PIL import Image
import io
import base64
import PyPDF2
try:
    from xml.etree.cElementTree import XML
except ImportError:
    from xml.etree.ElementTree import XML
import zipfile

In [0]:
from ocr.utils.expand_bounding_box import expand_bounding_box
from ocr.utils.sclite_helper import ScliteHelper
from ocr.utils.word_to_line import sort_bbs_line_by_line, crop_line_images
from ocr.utils.iam_dataset import IAMDataset, resize_image, crop_image, crop_handwriting_page
from ocr.utils.encoder_decoder import Denoiser, ALPHABET, encode_char, decode_char, EOS, BOS
from ocr.utils.beam_search import ctcBeamSearch

import ocr.utils.denoiser_utils
import ocr.utils.beam_search

importlib.reload(ocr.utils.denoiser_utils)
from ocr.utils.denoiser_utils import SequenceGenerator

importlib.reload(ocr.utils.beam_search)
from ocr.utils.beam_search import ctcBeamSearch



from ocr.paragraph_segmentation_dcnn import SegmentationNetwork, paragraph_segmentation_transform
from ocr.word_and_line_segmentation import SSD as WordSegmentationNet, predict_bounding_boxes
from ocr.handwriting_line_recognition import Network as HandwritingRecognitionNet, handwriting_recognition_transform
from ocr.handwriting_line_recognition import decode as decoder_handwriting, alphabet_encoding


In [0]:
from flask_ngrok import run_with_ngrok
from flask import Flask,request,jsonify

In [0]:
ctx = mx.gpu(0) if mx.context.num_gpus() > 0 else mx.cpu()

In [0]:
def resize_image(image, desired_size):
    ''' Helper function to resize an image while keeping the aspect ratio.
    Parameter
    ---------
    
    image: np.array
        The image to be resized.

    desired_size: (int, int)
        The (height, width) of the resized image

    Return
    ------

    image: np.array
        The image of size = desired_size

    bounding box: (int, int, int, int)
        (x, y, w, h) in percentages of the resized image of the original
    '''
    size = image.shape[:2]
    if size[0] > desired_size[0] or size[1] > desired_size[1]:
        ratio_w = float(desired_size[0])/size[0]
        ratio_h = float(desired_size[1])/size[1]
        ratio = min(ratio_w, ratio_h)
        new_size = tuple([int(x*ratio) for x in size])
        image = cv2.resize(image, (new_size[1], new_size[0]))
        size = image.shape
            
    delta_w = max(0, desired_size[1] - size[1])
    delta_h = max(0, desired_size[0] - size[0])
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)
            
    color = image[0][0]
    if color < 230:
        color = 230
    image = cv2.copyMakeBorder(image, top, bottom, left, right, cv2.BORDER_CONSTANT, value=float(color))
    crop_bb = (left/image.shape[1], top/image.shape[0], (image.shape[1] - right - left)/image.shape[1],
               (image.shape[0] - bottom - top)/image.shape[0])
    image[image > 230] = 255
    return image, crop_bb




MAX_IMAGE_SIZE_FORM = (1120, 800)
MAX_IMAGE_SIZE_LINE = (60, 800)
MAX_IMAGE_SIZE_WORD = (30, 140)


#this function takes in the img file path
def _pre_process_image(img_in,_parse_method):
    #im = cv2.imread(img_in, cv2.IMREAD_GRAYSCALE)
    im = img_in
    if np.size(im) == 1: # skip if the image data is corrupt.
        return None
    # reduce the size of form images so that it can fit in memory.
    if _parse_method in ["form", "form_bb"]:
        im, _ = resize_image(im, MAX_IMAGE_SIZE_FORM)
    if _parse_method == "line":
        im, _ = resize_image(im, MAX_IMAGE_SIZE_LINE)
    if _parse_method == "word":
        im, _ = resize_image(im, MAX_IMAGE_SIZE_WORD)
    img_arr = np.asarray(im)
    return img_arr


In [0]:
paragraph_segmentation_net = SegmentationNetwork(ctx=ctx)
paragraph_segmentation_net.cnn.load_parameters("models/paragraph_segmentation2.params", ctx=ctx)

paragraph_segmentation_net.hybridize()    


form_size = (1120, 800)
def paragraph_segmentation(images):
  predicted_bbs = []
  for i, image in enumerate(images):
    resized_image = paragraph_segmentation_transform(image, form_size)
    bb_predicted = paragraph_segmentation_net(resized_image.as_in_context(ctx))
    bb_predicted = bb_predicted[0].asnumpy()
    bb_predicted = expand_bounding_box(bb_predicted, expand_bb_scale_x=0.03,
                                           expand_bb_scale_y=0.03)
    predicted_bbs.append(bb_predicted)
  return predicted_bbs


In [0]:
segmented_paragraph_size = (700, 700)

def paragraph_segment_crop(images,predicted_bbs):
  paragraph_segmented_images = []
  for i, image in enumerate(images):
    bb = predicted_bbs[i]
    image = crop_handwriting_page(image, bb, image_size=segmented_paragraph_size)
    paragraph_segmented_images.append(image)
  return paragraph_segmented_images


In [0]:
word_segmentation_net = WordSegmentationNet(2, ctx=ctx)
word_segmentation_net.load_parameters("models/word_segmentation2.params")
word_segmentation_net.hybridize()


min_c = 0.1
overlap_thres = 0.1
topk = 600

def word_segmentation(paragraph_segmented_images):
  predicted_words_bbs_array = []
  for i, paragraph_segmented_image in enumerate(paragraph_segmented_images):
    predicted_bb = predict_bounding_boxes(
        word_segmentation_net, paragraph_segmented_image, min_c, overlap_thres, topk, ctx)

    predicted_words_bbs_array.append(predicted_bb)
  return predicted_words_bbs_array

In [0]:
def line_segmentation(predicted_words_bbs_array,paragraph_segmented_images):
  line_images_array = []
  for i, paragraph_segmented_image in enumerate(paragraph_segmented_images):
    predicted_bbs = predicted_words_bbs_array[i]
    line_bbs = sort_bbs_line_by_line(predicted_bbs, y_overlap=0.4)
    line_images = crop_line_images(paragraph_segmented_image, line_bbs)
    line_images_array.append(line_images)
  return line_images_array

In [0]:
handwriting_line_recognition_net = HandwritingRecognitionNet(rnn_hidden_states=512,
                                                             rnn_layers=2, ctx=ctx, max_seq_len=160)
handwriting_line_recognition_net.load_parameters("models/handwriting_line8.params", ctx=ctx)
handwriting_line_recognition_net.hybridize()


line_image_size = (60, 800)

def handwriting_line_recognition(line_images_array):
  character_probs = []
  for line_images in line_images_array:
    form_character_prob = []
    for i, line_image in enumerate(line_images):
        line_image = handwriting_recognition_transform(line_image, line_image_size)
        line_character_prob = handwriting_line_recognition_net(line_image.as_in_context(ctx))
        form_character_prob.append(line_character_prob)
    character_probs.append(form_character_prob)
  return character_probs

In [0]:
def get_arg_max(prob):
    '''
    The greedy algorithm convert the output of the handwriting recognition network
    into strings.
    '''
    arg_max = prob.topk(axis=2).asnumpy()
    return decoder_handwriting(arg_max)[0]

In [0]:
def decode_line(character_probs):
  decoded_images = {}
  for i, form_character_probs in enumerate(character_probs):
    text = []
    for j, line_character_probs in enumerate(form_character_probs):
      decoded_line_am = get_arg_max(line_character_probs)
      text.append(decoded_line_am)
    text = ','.join(text)
    decoded_images.update({i:text})
  return decoded_images

In [0]:
def decode_images(images):
  decoded = []
  for image in images:
    imgdata = base64.b64decode(image)
    image = Image.open(io.BytesIO(imgdata))
    image = cv2.cvtColor(np.array(image), cv2.COLOR_BGR2RGB)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    decoded.append(image)
  return decoded

In [0]:
def pdfToText(myfile):
  #print(myPdf.numPages)
  myPdf = PyPDF2.PdfFileReader(myfile)
  num_pages = myPdf.numPages
  count = 0
  text = ""
  while count < num_pages:
    pageObj = myPdf.getPage(count)
    count +=1
    text += pageObj.extractText()
  #print(text)
  return text

In [0]:
def decode(encoded):
  decoded = []
  for pdf in encoded:
    pdf = base64.b64decode(pdf)
    pdf = io.BytesIO(pdf)
    decoded.append(pdf)
  return decoded

In [0]:
WORD_NAMESPACE = '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}'
PARA = WORD_NAMESPACE + 'p'
TEXT = WORD_NAMESPACE + 't'


In [0]:
def get_docx_text(doc):
    document = zipfile.ZipFile(doc)
    xml_content = document.read('word/document.xml')
    document.close()
    tree = XML(xml_content)

    paragraphs = []
    for paragraph in tree.getiterator(PARA):
        texts = [node.text
                 for node in paragraph.getiterator(TEXT)
                 if node.text]
        if texts:
            paragraphs.append(''.join(texts))

    return '\n\n'.join(paragraphs)


In [0]:
app = Flask(__name__)
run_with_ngrok(app)
@app.route('/image-text/<uuid>',methods = ['POST','GET'])
def home(uuid):
  content = request.get_json(force=True)
  #print(content)
  images = content["images"]
  images_decoded = decode_images(images)
  processed = []
  for image in images_decoded:
    image = _pre_process_image(image,'form')
    processed.append(image)
  random.seed(1)
  predicted_bbs = paragraph_segmentation(processed)
  paragraph_segmented_images = paragraph_segment_crop(images = processed,predicted_bbs= predicted_bbs)
  predicted_words_bbs_array = word_segmentation(paragraph_segmented_images = paragraph_segmented_images)
  line_images_array = line_segmentation(predicted_words_bbs_array = predicted_words_bbs_array, paragraph_segmented_images = paragraph_segmented_images)
  character_probs = handwriting_line_recognition(line_images_array= line_images_array)
  decoded_images = decode_line(character_probs = character_probs)
  return jsonify({"uuid" : uuid, "text" : decoded_images})
@app.route('/pdf-text/<uuid>',methods = ['POST'])
def pdf_text(uuid):
  content = request.get_json(force=True)
  pdfs = content["pdfs"]
  pdfs_decoded = decode(pdfs)
  texts = {}
  i = 0
  for pdf in pdfs_decoded:
    pdf_converted = pdfToText(pdf)
    texts.update({i:pdf_converted})
    i = i+1
  return jsonify({"uuid":uuid,"text":texts})
@app.route('/word-text/<uuid>',methods = ['POST'])
def word_text(uuid):
  content = request.get_json(force=True)
  #print(content)
  docs = content["docs"]
  docs_decoded = decode(docs)
  texts = {}
  i = 0
  for doc in docs_decoded:
    doc_converted = get_docx_text(doc)
    texts.update({i:doc_converted})
    i = i+1
  return jsonify({"uuid":uuid,"text":texts})

In [0]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://c3551dd3.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [23/Jan/2020 03:52:30] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [23/Jan/2020 03:57:26] "POST /pdf-text/123 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jan/2020 04:07:25] "POST /pdf-text/123 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jan/2020 04:11:52] "POST /pdf-text/123 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jan/2020 04:13:30] "POST /pdf-text/123 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jan/2020 04:14:14] "POST /pdf-text/123 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jan/2020 04:14:53] "POST /pdf-text/123 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jan/2020 04:15:27] "POST /pdf-text/123 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jan/2020 04:17:09] "POST /pdf-text/123 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jan/2020 04:18:08] "POST /pdf-text/123 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jan/2020 04:20:31] "POST /pdf-text/123 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jan/2020 04:22:50] "POST